In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pt
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
import warnings
warnings.filterwarnings('ignore')

# **dataset loading**

In [ ]:
data=pd.read_csv('/content/sample_data/titanic (1).csv')

In [ ]:
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


***Selecting dependent and independent variables***

In [ ]:
X = data[['Pclass','SibSp','Parch','Fare']]
y = data.Survived

In [ ]:
X.head()

,Pclass,SibSp,Parch,Fare
0,3,1,0,7.2500
1,1,1,0,71.2833
2,3,0,0,7.9250
3,1,1,0,53.1000
4,3,0,0,8.0500


*test-train data*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# **Model building**

In [ ]:
model_svc = SVC(kernel='linear', C=1, gamma='auto')
model_svc.fit(X_train,y_train)

y_pred = model_svc.predict(X_test)

print('Accuracy Score:\n',accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred))
print('Classification Report:\n',classification_report(y_test,y_pred))

Accuracy Score:
 0.7094972067039106
Confusion Matrix:
 [[96 17]
 [35 31]]
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.85      0.79       113
           1       0.65      0.47      0.54        66

    accuracy                           0.71       179
   macro avg       0.69      0.66      0.67       179
weighted avg       0.70      0.71      0.70       179



*Use 5-Fold Cross Validation to generalise the model accuracy.*

In [ ]:
kf=KFold(n_splits=5)
score=cross_val_score(model_svc,X,y,cv=kf)

print("Cross Validation Scores are: {}\n".format(score))

print("Average of Cross Validation Scores: {}".format(score.mean()))

Cross Validation Scores are: [0.61452514 0.65730337 0.66292135 0.7247191  0.74719101]

Average of Cross Validation Scores: 0.6813319942250958


# **Random search**

In [ ]:
search_space ={'C': [0.1, 1, 10, 100, 1000],
     'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
     'kernel': ['rbf']}

In [ ]:
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=search_space,verbose=2, n_jobs=-1,
                            n_iter=50)
random_result = random_search.fit(X_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Score:  71.21146459174629
Best Params:  {'kernel': 'rbf', 'gamma': 1, 'C': 10}


In [ ]:
model_svc = SVC(kernel='rbf', C=1000, gamma=0.001)
model_svc.fit(X_train,y_train)

y_pred = model_svc.predict(X_test)

print('Accuracy Score:\n',accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred))
print('Classification Report:\n',classification_report(y_test,y_pred))

Accuracy Score:
 0.6983240223463687
Confusion Matrix:
 [[94 19]
 [35 31]]
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.83      0.78       113
           1       0.62      0.47      0.53        66

    accuracy                           0.70       179
   macro avg       0.67      0.65      0.66       179
weighted avg       0.69      0.70      0.69       179



*Use 5-Fold Cross Validation to generalise the model accuracy.*


In [ ]:
kf=KFold(n_splits=5)
score=cross_val_score(model_svc,X,y,cv=kf)

print("Cross Validation Scores are: {}\n".format(score))

print("Average of Cross Validation Scores: {}".format(score.mean()))

Cross Validation Scores are: [0.63687151 0.64606742 0.73033708 0.71348315 0.74157303]

Average of Cross Validation Scores: 0.6936664365074383


# **Hyperopt**

In [ ]:
def hyperparameter_tuning(params):
    clf = SVC(**params)
    acc = cross_val_score(clf, X, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [ ]:
search_space = {
    "C": hp.choice("C", [0.1, 1, 10, 100, 1000]),
    "gamma": hp.choice("gamma", [1, 0.1, 0.01, 0.001, 0.0001]),
    "kernel": hp.choice("kernel", ['rbf']),
}

trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = search_space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 50/50 [00:56<00:00,  1.13s/it, best loss: -0.7060134329295085]
Best: {'C': 4, 'gamma': 1, 'kernel': 0}


In [ ]:
from sklearn.svm import SVC
model_svc = SVC(kernel='rbf', C=1000, gamma=0.1)
model_svc.fit(X_train,y_train)

y_pred = model_svc.predict(X_test)

print('Accuracy Score:\n',accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred))
print('Classification Report:\n',classification_report(y_test,y_pred))

Accuracy Score:
 0.7094972067039106
Confusion Matrix:
 [[93 20]
 [32 34]]
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.82      0.78       113
           1       0.63      0.52      0.57        66

    accuracy                           0.71       179
   macro avg       0.69      0.67      0.67       179
weighted avg       0.70      0.71      0.70       179



*Use 5-Fold Cross Validation to generalise the model accuracy.*

In [ ]:
from sklearn.model_selection import cross_val_score,KFold

kf=KFold(n_splits=5)
score=cross_val_score(model_svc,X,y,cv=kf)

print("Cross Validation Scores are: {}\n".format(score))

print("Average of Cross Validation Scores: {}".format(score.mean()))

Cross Validation Scores are: [0.67039106 0.63483146 0.75280899 0.70224719 0.7752809 ]

Average of Cross Validation Scores: 0.7071119201556713


:

# *  **Tuning method**: Random Search

**Cross validation score :0.694**



---


C           :10

---


Gamma       :1

---


Kernal:rbf

---


# *   **Tuning method:**Hyperopt

**Cross validation score:0.707**

---


C           : 1000

---


Gamma       :0.1

---


Kernal:rbf

---








